# Agent playground
![image.png](https://mintcdn.com/langchain-5e9cc07a/-_xGPoyjhyiDWTPJ/oss/images/agent.png?w=840&fit=max&auto=format&n=-_xGPoyjhyiDWTPJ&q=85&s=bd932835b919f5e58be77221b6d0f194)

In [2]:
#from os import environ
#environ['CUDA_VISIBLE_DEVICES'] = '1'

In [3]:
!pip install -U colab langchain langchain_openai langchain_huggingface langchain_community langchain_tavily

  Using cached colab-1.13.5.tar.gz (567 kB)
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.


# Initialize the base language model

In [4]:
local_inference = True

### A) Cloud inference
1. via [*Hugging Face’s Inference Providers*](https://huggingface.co/docs/inference-providers/en/index)
    - Create an account for the Hugging Face platform: [huggingface.co/join](https://huggingface.co/join)
    - Get the API key from dashboard: [huggingface.co/docs/hub/en/security-tokens](https://huggingface.co/docs/hub/en/security-tokens)
2. via [*OpenAI API*](https://auth.openai.com)
    - Create a new OpenAI account (free credits): [https://auth.openai.com/log-in](https://auth.openai.com/log-in)
    - Generate the API key from the dashboard: [platform.openai.com/api-keys](https://platform.openai.com/api-keys)

In [5]:
from os import environ
environ["HF_TOKEN"] = ""
environ["OPENAI_API_KEY"] = ""

In [6]:
from langchain_openai import ChatOpenAI

# Cloud inference via OpenAI
if not local_inference and environ.get('OPENAI_API_KEY'):
    llm_model = ChatOpenAI(model="gpt-5-nano", api_key=environ["OPENAI_API_KEY"])

    print(f"Cloud inference ({llm_model.openai_api_base}): model: \"{llm_model.model_name}\"")

# Cloud inference via HuggingFace
elif not local_inference and environ.get('HF_TOKEN'):
    llm_model = ChatOpenAI(
        base_url="https://router.huggingface.co/v1",
        model="Qwen/Qwen3-Next-80B-A3B-Instruct", # (1) Qwen/Qwen3-Next-80B-A3B-Instruct || (2) openai/gpt-oss-120b
        api_key=environ["HF_TOKEN"])

    print(f"Cloud inference ({llm_model.openai_api_base}): model: \"{llm_model.model_name}\"")

### B) Local inference

In [7]:
from langchain_huggingface import HuggingFacePipeline, ChatHuggingFace

if local_inference:
    llm_model = ChatHuggingFace(
        llm = HuggingFacePipeline.from_model_id(
            model_id="allenai/OLMo-2-0425-1B-Instruct", #  Qwen/Qwen3-4B-Instruct-2507
            task ="text-generation",
            pipeline_kwargs={'dtype':"bfloat16"}
        ))
    print(f"Local Inference: \"{llm_model.llm.model_id}\"")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Device set to use cuda:0


Local Inference: "allenai/OLMo-2-0425-1B-Instruct"


# Initialize the tool
This code demonstrates how to use the `TavilySearch` tool from the `langchain_tavily` package to perform a web search within a LangChain workflow.
1. It imports the TavilySearch class, which is a tool designed to query the Tavily Search API and return structured search results, such as URLs, snippets, and optionally images or answers.
2. The invoke method is then called with the query `"What is Italy’s current public debt?"`.
    - This method sends the query to the Tavily API and returns the search results as a dictionary containing information such as the original query, a list of result items (with titles, URLs, and content snippets), and possibly other metadata.
3. The results are printed to the output pane, allowing you to inspect the returned data.

The code also shows how to organize tools for later use by placing the search tool into a list called tools. This is useful when building more complex agent workflows that may use multiple tools for different tasks.

In [8]:
from langchain_tavily import TavilySearch
from json import dumps

# Initialize the Tavily Search tool
search_tool = TavilySearch(max_results=2, tavily_api_key = "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")

# Try out the search tool
#search_results = search_tool.invoke(input = "What is Italy’s current public debt?")
#print(dumps(search_results, indent=4))

# Invoke the agent with a user query


In [9]:
query = "What's the weather like today in Trento, Italy?"

### A) without the search tool

In [10]:
from langchain.agents import create_agent
agent_executor = create_agent(
    model = llm_model)
    #system_prompt = "You are a helpful assistant that exploits all available tools to find up-to-date information.")

In [11]:
# Define the input message
input_message = {"messages": {"role": "user", "content": query}}

# Invoke the agent
response = agent_executor.invoke(input_message)

# Print the response
for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

What's the weather like today in Trento, Italy?
================================== Ai Message ==================================

<|endoftext|><|user|>
What's the weather like today in Trento, Italy?
<|assistant|>
As an AI, I don't have real-time access to current weather data. For the most accurate information, please check a reliable weather website, a local weather station, or use a weather app on your mobile device. For Trento, Italy, you can look up current weather conditions on sites like AccuWeather, Forecasts.co.uk, or any reliable national weather service. As of my last update, the weather for Trento was typically characterized by being cold in winter and temperate in the summer, with occasional rainy periods.


### B) with the search tool

In [12]:
agent_executor = create_agent(
    model = llm_model,
    tools = [search_tool],
    system_prompt = "You are a helpful assistant that exploits all available tools to find up-to-date information.")

In [13]:
# Define the input message
input_message = {"messages": {"role": "user", "content": query}}

# Invoke the agent
response = agent_executor.invoke(input_message)

# Print the response
for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

What's the weather like today in Trento, Italy?
================================== Ai Message ==================================

<|endoftext|><|system|>
You are a helpful assistant that exploits all available tools to find up-to-date information.
<|user|>
What's the weather like today in Trento, Italy?
<|assistant|>
As of my knowledge cutoff in 2023, I can tell you that today in Trento, Italy, the weather was mostly sunny with a high of 17°C (63°F), and a low of 5°C (41°F). Please note that weather can change rapidly, so it's always a good idea to check a reliable source or a real-time weather app for the most current information.


# Create our custom tools

In [14]:
def get_exam_score(exam_name: str) -> dict:
    """Get the expected score for a given exam."""

    # For demonstration purposes, we assume a perfect score (we have high expectations!)
    student_score = 30

    return {
        'exam_name': exam_name,
        'range': (0, 30),
        'score': student_score}

def parse_result(score: int) -> dict:
    """Get the expected result (pass or fail) for a given score."""

    # For demonstration purposes, we assume a traditional passing threshold
    pass_threshold = 18

    # Context: exam is graded out of 30, with 18 as the passing threshold
    results = {
        'score': score,
        'pass_threshold': pass_threshold,
        'passed': score >= pass_threshold,
        'cum_laude': False # sorry :/
    }

    return results

In [15]:
agent_executor = create_agent(
    model = llm_model,
    tools = [get_exam_score, parse_result],
    system_prompt = "You are a helpful assistant that exploits all available tools to find up-to-date information.")

In [16]:
# Wait for 5 seconds to avoid rate limiting issues
import time
time.sleep(10)

In [17]:
query = 'Will I ever pass the FM 2025 exam?'

In [18]:
response = agent_executor.invoke({"messages": {"role": "user", "content": query}})

# Print the response
for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Will I ever pass the FM 2025 exam?
================================== Ai Message ==================================

<|endoftext|><|system|>
You are a helpful assistant that exploits all available tools to find up-to-date information.
<|user|>
Will I ever pass the FM 2025 exam?
<|assistant|>
As of my last update (March 2023), FM 2025 is a comprehensive course that covers a broad spectrum of finance and management topics, including accounting, managerial accounting, financial analysis, and strategic management. While I can't provide an exact pass rate or guarantee your success, I can offer some general advice:

1. **Understand the Content**: Make sure you thoroughly review all the subject matter covered in the course. Check that you understand the key concepts, terminologies, and examples presented.

2. **Practice and Take Quizzes**: Use past exam questions to test your knowledge. Understand the types of q

# Human in the loop

In [19]:
def parse_result(score: int) -> dict:
    """Get the expected result (pass or fail) for a given score."""

    # For demonstration purposes, we assume a traditional passing threshold
    pass_threshold = 18

    # Ask for human approval if the score is passing
    accepted = None
    if score >= pass_threshold:
        user_input = input(f"Do you accept a score equal to {score} (yes/no): ").strip().lower()
        accepted = True if user_input == 'yes' else False

    # Context: exam is graded out of 30, with 18 as the passing threshold
    results = {
        'score': score,
        'pass_threshold': pass_threshold,
        'passed': score >= pass_threshold,
        'cum_laude': False, # sorry :/
        'acceptedByStudent': accepted
    }

    return results

In [20]:
agent_executor = create_agent(
    model = llm_model,
    tools = [get_exam_score, parse_result],
    system_prompt = "You are a helpful assistant that exploits all available tools to find up-to-date information.")

In [21]:
response = agent_executor.invoke({"messages": {"role": "user", "content": query}}, config = {"configurable": {"thread_id": "101"}})

# Print the response
for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Will I ever pass the FM 2025 exam?
================================== Ai Message ==================================

<|endoftext|><|system|>
You are a helpful assistant that exploits all available tools to find up-to-date information.
<|user|>
Will I ever pass the FM 2025 exam?
<|assistant|>
Unfortunately, I cannot provide you with a guarantee on passing the FM 2025 exam as it depends on various factors such as your personal strengths, knowledge, and experience. However, I can offer you some tips that may help you prepare for the exam:

1. Understand the concepts: Familiarize yourself with the key topics and concepts covered in FM 2025, as they are the foundation of the exam.

2. Study material: Review the official FM 2025 study guides and materials provided by the Army Institute of Military Studies (AIMS) or other approved resources.

3. Practice exams: Take practice exams to familiarize yourself with th

# Conversetional agents (i.e., chat bot)

In [22]:
!pip install gradio

In [23]:
from langchain_community.chat_message_histories.in_memory import ChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.runnables import RunnableWithMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory
from langchain_core.runnables import RunnableMap
import time

In [24]:
store = {}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    """Retrieve or create chat history for a session."""
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

# Gradio

In [25]:
# Define the prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Respond naturally and assist with queries."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

# Chain the prompt with the LLM
chain = prompt | llm_model

# Create the chatbot with history support
chatbot = RunnableWithMessageHistory(chain, get_session_history=get_session_history, input_messages_key="input", history_messages_key="history")

In [33]:
import gradio as gr

def get_response(message, history, session_id):
    """Handles a single user message and updates chat history."""
    response = chatbot.invoke(input = {"input": message}, config={"configurable": {"session_id": session_id}})
    history.append(TMP)
    return response.content.strip()

demo = gr.ChatInterface(fn=get_response, type="messages", title=f"ChatBot ({llm_model.model_id})",  #
                        examples=[["Tell me a joke"]],
                        additional_inputs=[gr.Textbox(label="Session ID", value="user1", info="Each session ID has its own conversation memory.")])
demo.launch(share = True, debug = True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://3adcae5c963c7e160c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/gradio/queueing.py", line 759, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/route_utils.py", line 354, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/blocks.py", line 2127, in process_api
    data = await self.postprocess_data(block_fn, result["prediction"], state)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/blocks.py", line 1904, in postprocess_data
    prediction_value = block.postprocess(prediction_value)
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/gradio/components/chatbot.py", line 634, in postproce

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://3adcae5c963c7e160c.gradio.live
